# Motivação
Até o meio do século XX, todos os métodos de criptografia se baseavam num mesmo mecanismo. Dos mais simples, como a cifra de césar, até coisas como o blowfish, todos estes algoritmos, chamados de **Algoritmos de Chave Simétrica**, trabalham da mesma forma: o remetente realiza uma transformação de uma informação em função de uma chave $K$ e transmitem o resultado. O receptor, em posse da mesma chave, realiza uma transformação inversa para conseguir acesso à informação. Esse método gera dois grandes problemas: 

1. Todos os participantes da "conversa" precisam ter a chave $K$, que precisa ser transmitida de forma segura entre eles, que não necessariamente é fácil, mesmo tendo acesso físico uns aos outros.

2. Considerando várias chaves para vários parceiros de conversa, a gerência de diferentes chaves também precisa presar pela segurança e distribuição de cada uma delas, algo extremamente complicado em grande escala.

Tendo em mente estes problemas, [James H. Ellis](https://en.wikipedia.org/wiki/James_H._Ellis) teorizou um sistema de criptografia que usasse um par de chaves: uma pública, que qualquer um pode usar para encriptar uma mensagem a ser enviada ao dono das chaves, e uma privada, conhecida apenas pelo recipiente, que é a chave para descriptografar a mensagem. Dessa forma, a chave pública pode ser distribuída livremente, sem riscos de quebra de privacidade da comunicação. O primeiro algoritmo baseado nesse sistema (chamado, adequadamente, de **Criptografia de Chave Pública**), foi desenvolvido por [Clifford Cocks](https://en.wikipedia.org/wiki/Clifford_Cocks) em 1970, e mantido em segredo pelo Quartel General de Comunicações do reino unido. Uma variação do algoritmo de Cocks foi desenvolvido independentemente por três pesquisadores do MIT  e publicado em 1978. Por isso, acabou ficando conhecido como RSA, as iniciais dos três.

# Base Matemática
## Conceitos Básicos
Matemáticamente, o RSA se baseia em um campo de estudo chamado Teoria dos números, especialmente a parte que lida com números primos. Para entendimento dos teoremas, o leitor precisa saber que 

1. Existe uma operação modulo, tal que $a \pmod{n}$ é igual ao resto da operação $a \div n$.

1. Sendo $a$, $b$  e $n$ inteiros, a notação $a \equiv b \pmod{n}$ (Lê-se $a$ e $b$ são **congruentes** módulo $n$) indica que $a\pmod{n}$ e $b\pmod{n}$ resultam no mesmo número.

1. Dizer que $a$ é **coprimo** de $b$ significa que o maior divisor comum de $a$ e $b$ é $1$ ($\gcd{a, b} = 1$)

1. A função totiente de Euler ($\phi(N)$) é equivalente ao número de inteiros menores que $N$ que são coprimos com $N$.

## Teoremas
### Teorema de Euler
Este teorema é descrito como: $$x^{\phi(N)} \equiv 1\pmod{N}$$ para qualquer $x$ coprimo a $N$, ambos inteiros positivos. Este é o caso geral do teorema principal dessa seção:

### Teorema Fundamental do RSA
$$x^{(p-1)(q-1)}\equiv1\pmod{pq}$$
Para quaisquer primos $p$ e $q$ e um $x$ coprimo a ambos.

A segurança reside no fato de que o único jeito de, sem saber a chave privada da comunicação, decriptar as mensagens, seria descobrir $p$ e $q$. Como a parte pública do RSA é o produto $n=pq$, e outro valor dependente $p$ e $q$, como veremos a seguir, o jeito de de descobri-los seria realizando a decomposição em valores primos de $n$, problema que, apesar de não ter sido provado como de complexidade NP, não possui nenhuma solução publicada que o resolva em tempo exponencial.

(As provas dessa seção podem ser encontradas no livro *Números Inteiros e Criptografia RSA*, citado na biografia)

# O Algoritmo
O RSA em si é divido em 3 grandes partes:
## Criação de chaves
Primeiro, são gerados dois números primos distintos. Os metódos de geração variam e não fazem parte da especificação do algoritmo. De forma geral apresentam seguinte forma:
1. Gerar um número, ou de forma completamente aleatória, ou usando uma função que gere uma grande quantidade de números primos, estatisticamente (eg. [Números de Mersenne](https://en.wikipedia.org/wiki/Mersenne_prime))

2. Checar se o número é primo, usando algum teste de primalidade. Caso negativo, repetir o passo 1.

Para os exemplos a seguir usaremos dois primos simples:


In [12]:
p, q = 13, 17

Em posse desses dois números, precisamos gerar as chaves em si. O primeiro passo é descobrir $e$ e $d$ tal que $ed \equiv 1 \pmod{\phi}$ sendo $\phi = (p − 1)(q − 1)$.
Escolhemos $e$ entre os coprimos de $\phi$ menores que o mesmo. Tendo esse número, executamos o algoritmo euclidiano estendido para descobrir $d$, seu inverso multiplicativo modular.
A chave pública será $(e, n)$ e a privada $(d, n)$ sendo $n = pq$.

Em código:


In [13]:
import random
from math import gcd

def euclidianoEstendido(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = euclidianoEstendido(b % a, a)
    return (g, x - (b // a) * y, y)

def inversoModular(a, m):
    g, x, y = euclidianoEstendido(a, m)
    return x % m

def gerarChaves(p, q):
    #n = pq
    n = p * q
    #Phi é o totiente n
    phi = (p-1) * (q-1)
    while True:
    #Geramos um e entre 1 e phi
        e = random.randrange(3, phi)
        #Checamos se e e phi são coprimos, descobrindo seu mdc
        mdc = gcd(e, phi)
        if(mdc == 1):
            #Euclidiano Estendido para descobrir d
            d = inversoModular(e, phi)
            if(d != e):
                break
    
    #Retorna o par de chaves
    #Chave pública é (e, n) e Chave privada é (d, n)

    return ((e, n), (d, n))


chavePublica, chavePrivada = gerarChaves(p, q)
print("Chave publica:" + str(chavePublica))
print("Chave privada:" + str(chavePrivada))

Chave publica:(145, 221)
Chave privada:(49, 221)


## Encriptação

A encriptação é o processo de dividir a mensagem em pacotes $m$ com valor menor que $n$ e realizar para cada bloco a operação $m^{e} \pmod{n}$. Nesse caso, usaremos os caracteres da mensagem como blocos*:

In [24]:
frase = "Plus Ultra"

def encriptar(chave, mensagem):
    #Separar os componentes da chave
    e, n = chave
    #Converter cada caractere em um número usando a^b mod m
    textoEncriptado = [(ord(letra) **e) % n for letra in mensagem]
    #Retornar o array
    return textoEncriptado

mensagemEncriptada = encriptar(chavePublica, frase)
print("Mensagem Original: '"+frase+"' \nMensagem Encriptada: ")
print(mensagemEncriptada)


Mensagem Original: 'Plus Ultra' 
Mensagem Encriptada: 
[80, 108, 117, 115, 32, 85, 108, 116, 114, 97]


*_o uso de caracteres como blocos é altamente desencorajado em aplicações reais, por deixar o criptotexto vulnerável a decodificação por uso de frequência de letras._

## Decriptação
Na decriptação recebe-se um array de pacotes encriptados $c$ e realiza-se para cada bloco a operação $c^{d} \pmod{n}$. Depois juntamos os blocos em uma string:

In [23]:
def decriptar(chave, cifra):
    #Separar os componentes da chave
    d, n = chave
    
    #Converter cada número em caractere usando a^b mod m
    texto = [chr((bloco ** d) % n) for bloco in cifra]

    #Retornar o string
    return ''.join(texto)

mensagemRecuperada = decriptar(chavePrivada, mensagemEncriptada)
print("Mensagem decriptada: '" + mensagemRecuperada+ "'")


Mensagem decriptada: 'Plus Ultra'


# Bibliografia
* [Curso de fundamentos de segurança de computadores - UTA - aula 44](https://www.cs.utexas.edu/users/byoung/cs361/lecture44.pdf)
* [Paper de James Ellis sobre Encriptação Não-secreta](https://web.archive.org/web/20030610193721/http://jya.com/ellisdoc.htm)
* [Números Inteiros e Criptografia RSA - S. C. Coutinho](https://loja.sbm.org.br/index.php/numeros-inteiros-e-criptografia-rsa.html)
* [Public-key cryptography - Wikipedia](https://en.wikipedia.org/wiki/Public-key_cryptography)
* [RSA problem - Wikipedia](https://en.wikipedia.org/wiki/RSA_problem)
* Various wikipedia pages